In [1]:
import cPickle as pickle
import pandas as pd
from pandas import DataFrame
base_path = '/home/hareesh/Research/big_data/project/airbnb/'

In [2]:
df_listings = pd.read_csv(base_path+'inside_airbnb/toronto/listings.csv')
df_listings.shape

(12029, 95)

In [3]:
with open('toronto_listing_amenities', 'rb') as f:
    amenity_dict_available = pickle.load(f)
    amenity_dict_not_available = pickle.load(f)
    page_saved_dict = pickle.load(f)
    
len(page_saved_dict)

9795

In [4]:
# 9795 listings out of 12029 were scraped - since some listings were removed from airbnb
# At other times, selenium failed in scraping the pages
sample_id = 3833863
print 'Amenities available at sample listing:\n', amenity_dict_available[sample_id]

Amenities available at sample listing:
[u'Free parking on premises', u'Elevator in building', u'Internet', u'Family/kid friendly', u'Wireless Internet', u'Buzzer/wireless intercom', u'Kitchen', u'Doorman', u'Wheelchair accessible', u'Cable TV', u'Hot tub', u'Gym', u'Pool', u'TV', u'Dryer', u'Washer', u'Essentials', u'Shampoo', u'Heating', u'Air conditioning']


In [5]:
print 'Amenities not available at sample listing:\n', amenity_dict_not_available[sample_id]

Amenities not available at sample listing:
[u'Pets allowed', u'Suitable for events', u'Smoking allowed', u'Indoor fireplace', u'Breakfast', u'Laptop friendly workspace', u'Iron', u'Hangers', u'Hair dryer', u'Private living room', u'Private entrance']


In [6]:
print 'Number of users who have saved this sample listing:', page_saved_dict[sample_id]

Number of users who have saved this sample listing: 4


In [7]:
# Create a dataframe for amenities: 1 if present,0 otherwise
column_names = amenity_dict_available[sample_id] + amenity_dict_not_available[sample_id] + [u'Paid parking off premises', u'Free parking on street', 'num_page_saved']

df_amenities = pd.DataFrame(0, columns=column_names, index=amenity_dict_available.keys())
for i in amenity_dict_available.keys():
    df_amenities.ix[i,amenity_dict_available[i]] = 1
    df_amenities.ix[i, 'num_page_saved'] = page_saved_dict[i]

In [8]:
df_amenities.head()

,Free parking on premises,Elevator in building,Internet,Family/kid friendly,Wireless Internet,Buzzer/wireless intercom,Kitchen,Doorman,Wheelchair accessible,Cable TV,...,Breakfast,Laptop friendly workspace,Iron,Hangers,Hair dryer,Private living room,Private entrance,Paid parking off premises,Free parking on street,num_page_saved
3833863,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,4
13795337,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,0,1,0,0,20
7602191,1,0,0,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,8
14647315,0,0,1,0,1,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,22
7897109,0,1,1,0,1,1,1,0,0,1,...,0,1,1,1,1,0,0,0,0,324


In [9]:
# Only Columns required in the analysis are retained
req_cols = ['id', 'name', 'summary', 'latitude', 'longitude', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 
'interaction', 'house_rules', 'host_id', 'host_since', 'host_about', 'host_response_time', 'host_response_rate'
'host_is_superhost', 'guests_included', 'extra_people', 'minimum_nights', 'number_of_reviews',
'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'cancellation_policy',
'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month']

df_listings = df_listings.ix[:,req_cols]

In [10]:
# Merge the 2 dataframe to a single dataframe with the req columns
df_amenities.reset_index(inplace=True)
df_listings = pd.merge(left = df_listings, right=df_amenities, how='inner', left_on='id', right_on='index')
df_listings.drop(['index'], inplace=True, axis=1)
df_listings.shape

(9795, 69)

In [11]:
# Save dataframe as CSV - can be loaded for data exploration later
df_listings.to_csv('listings_with_amenities_dataframe.csv', index=False)